# Unstructured Grid Overview

---

The goal of this notebook is to provide a brief overview of unstructured grids and provide a teaser of plotting with the UXarray package.
Contents:
1. Structured vs. Unstructured Grids
2. Structured Grids
3. Unstructured Grids
4. Unstructured Grids Benefits
5. Why UXarrary?
6. Loading and plotting a mesh with UXarray

## Structured vs Unstructured Grids


It is important to understand this difference, before diving into unstructured grid, as it is the main difference between the two types of grids:

A structured grid is well-ordered, with a simple scheme used to label elements and identify neighbors, while an unstructured grid allows elements to be joined in any manner, requiring special lists to identify neighboring elements.

Note that the focus here is on two dimensional grids in the climate and weather context, but the same concepts apply to three dimensional grids.

# Structured Grid
A few advantages of structured grids are:
- Uniform Representation: Simplifies numerical methods and enhances result interpretation.
  
- Efficient Numerics: Well-suited for finite-difference schemes, ensuring computational efficiency.
  
- Simplified Interpolation: Straightforward interpolation facilitates integration of observational data and model outputs.
  
- Boundary Handling: Ideal for regular boundaries, easing implementation of boundary conditions.
  
- Optimized Parallel Computing: Regular structure supports efficient parallel computing for scalability.

This code below uses xarray to create a structured mesh for temperature data interpolation. 

Given a set of coordinates and data points, the code uses numpy's meshgrid to generates a structured grid. The temperature data is then interpolated onto this grid, creating a smooth representation. Xarray is leveraged to organize the gridded data into a dataset, facilitating easy manipulation and visualization. The resulting plot showcases the data on this structure mesh, providing a clearer understanding of temperature variations across defined longitude and latitude ranges.

There are many other libraries and ways to create a structured grids.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Define coordinates
lon = np.array([-124.0, -122.0, -120.0])
lat = np.array([37.0, 37.0, 35.0])

# Define data
data = np.array([20, 22, 25])

# experiment with the number of points in the grid
# Try 3, 5, 10, 20, 50, 100, 1000
grid_points = 50 # 50x50 grid 

# Create a grid of coordinates
lon_grid, lat_grid = np.meshgrid(np.linspace(lon.min(), lon.max(), grid_points),
                                 np.linspace(lat.min(), lat.max(), grid_points))

# Interpolate the data onto the grid
temperature_grid = np.interp(lon_grid, lon, data)

# Create an xarray dataset
ds = xr.Dataset()
ds['temperature'] = (['lat', 'lon'], temperature_grid)
ds['lon'] = lon_grid[0, :]
ds['lat'] = lat_grid[:, 0]

# Plot the data
plt.pcolormesh(ds['lon'], ds['lat'], ds['temperature'], shading='auto')
plt.colorbar()
plt.title('Temperature')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


# Unstructured Grids

There are a variety of libraries and conventions for creating unstructured grids. Here we use a very basic standard python approach to showcase an unstructured grid: 

The code generates an unstructured grid over a rectangular domain defined by latitude and longitude ranges. It creates a set of points with slight irregularities and performs Delaunay triangulation using matplotlib.tri.Triangulation. The resulting unstructured mesh is visualized with triangular elements using plt.triplot, with original points plotted as blue circles, demonstrating the adaptability of unstructured grids in representing irregular terrains in climate science simulations.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as mtri

# Define the rectangular domain
lat_range = [35.0, 37.0]
lon_range = [-124.0, -120.0]

# Create a set of points with a little irregularity
n = 5  # Number of points
lats = np.linspace(lat_range[0], lat_range[1], n) + np.random.uniform(-0.5, 0.1, n)
lons = np.linspace(lon_range[0], lon_range[1], n) + np.random.uniform(-0.4, 0.2, n)
lons, lats = np.meshgrid(lons, lats)

# Perform triangulation
triang = mtri.Triangulation(lons.flatten(), lats.flatten())

# Most basic unstructured meshes need x and y coordinates for each triangle and connectivity information
print("Shape of triangles: ", triang.triangles.shape)
print(("x coordinate of first triangle: ", triang.x[triang.triangles[0]]))
print(("y coordinate of first triangle: ", triang.y[triang.triangles[0]]))
print("connectivity of first triangle: ", triang.triangles[0])

# Plot the result
plt.triplot(triang, marker='o')
plt.plot(lons, lats, 'bo')  # Plot the original points
plt.show()

Note that we don't add data to the above grid, manipulating the data on an unstructured grid is a bit more complicated than on a structured grid and often requires special interpolation techniques to get the data on the grid.

# Unstructured Grid Benefits
- Adaptability to complex geometries: Fits intricate shapes and boundaries

- Often runs faster than structured grids: Requires fewer elements to achieve similar accuracy
  
- Local refinement: Concentrates resolution on areas of interest
  
- Flexibility in element types: Accommodates various element shapes
  
- Efficient parallelization: Scales well to multiple processors
  
- Suitability for dynamic simulations: Adapts to changing conditions

# Why UXarray?

- Bring standardization to unstructured mesh support for climate data analysis and visualization.

- Adherence to the UGrid specification for compatibility with UGrid-compliant tools and data sources.

- Optimized data structures and algorithms for handling large and complex unstructured datasets.

- Simplified data access using Xarray's familiar data structures and operations. 

- Enhanced Interoperability and Community Collaboration.

- One interface for a variety of unstructured grid formats.

The above example uses triangulation to create an unstructured grid. However, there are many other ways to create unstructured grids. All have very specific conventions and formats. UXarray is a library that can read and write many of these formats. The specific focus of UXarray is to provide a common interface to access and manipulate unstructured grids in the climate science community.

## Loading and plotting a mesh with UXarray

The code below loads an MPAS OCEAN mesh and plots it using the UXarray library plot function.
This mesh

In [ ]:
import uxarray as ux


In [ ]:
import uxarray as ux
import requests

small_file_480km = requests.get(
            "https://web.lcrc.anl.gov/public/e3sm/inputdata/share/meshes/mpas/ocean/oQU480.230422.nc"
        ).content

ds_small_480km = ux.open_dataset(small_file_480km, small_file_480km)
print("The mesh contains:", ds_small_480km.uxgrid)

# Plot data on a mesh with UXarray

In [ ]:
ds_small_480km.uxgrid.plot(title="MPAS OCEAN Unstructured Mesh Plot", height=350, width=700)

Now check the variables in the mesh dataset and plot one of the variables:

In [ ]:
ds_small_480km

In [ ]:
ds_small_480km['bottomDepth'].plot.polygons(title="Bottom Depth Polygon Plot", height=350, width=700)

This notebook serves as an introduction to unstructured grids and UXarray. For more information, please visit the UXarray documentation at https://uxarray.readthedocs.io/en/latest/ and specifically see the example section: https://uxarray.readthedocs.io/en/latest/examples.html